In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 

import FINE.spagat.utils as spu
import FINE.spagat.RE_representation_utils as RE_rep_utils
from FINE.spagat.RE_representation import represent_RE_technology

In [3]:
DATA_PATH = 'C:/Users/s.patil/Documents/code/fine/examples/SPAGAT/RE_representation/InputData/'
shapefile = os.path.join(DATA_PATH, 'Shapefiles/CZE.shp')
GRIDDED_WIND_DATA_PATH = os.path.join(DATA_PATH, 'CZE_wind.nc4') 

In [4]:
represented_re_ds = represent_RE_technology(gridded_RE_ds = GRIDDED_WIND_DATA_PATH,
                                            CRS_attr = 'xy_reference_system',
                                            shp_file = shapefile,
                                            n_timeSeries_perRegion = 5,
                                            capacity_var_name='capacity',
                                            capfac_var_name='capfac',
                                            longitude='x', 
                                            latitude='y',
                                            time='time',
                                            index_col='e-id', 
                                            geometry_col='geometry',
                                            linkage='average')

Number of time series in 39_cz: 364
Number of time series in 40_cz: 213
elapsed time for represent_RE_technology: 0.08 minutes


In [5]:
represented_re_ds

<xarray.Dataset>
Dimensions:     (TS_ids: 5, region_ids: 2, time: 8760)
Coordinates:
  * region_ids  (region_ids) object '39_cz' '40_cz'
  * TS_ids      (TS_ids) <U4 'TS_0' 'TS_1' 'TS_2' 'TS_3' 'TS_4'
  * time        (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    capacity    (region_ids, TS_ids) float64 5.124e+06 1.327e+07 ... 1.359e+06
    capfac      (time, region_ids, TS_ids) float64 0.06577 0.004588 ... 0.0 0.0